In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kanichezhiyan","key":"c29d206a462c7b897fe544ba7dfb07a8"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d jayaprakashpondy/medicinal-plant-dataset

 97% 199M/206M [00:03<00:00, 112MB/s]
100% 206M/206M [00:03<00:00, 60.8MB/s]


In [ ]:
!unzip /content/medicinal-plant-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/train/Mallotus barbatus/45.JPG  
  inflating: Dataset/train/Mallotus barbatus/46.JPG  
  inflating: Dataset/train/Mallotus barbatus/47.JPG  
  inflating: Dataset/train/Mallotus barbatus/48.JPG  
  inflating: Dataset/train/Mallotus barbatus/49.JPG  
  inflating: Dataset/train/Mallotus barbatus/5.JPG  
  inflating: Dataset/train/Mallotus barbatus/50.JPG  
  inflating: Dataset/train/Mallotus barbatus/51.JPG  
  inflating: Dataset/train/Mallotus barbatus/52.JPG  
  inflating: Dataset/train/Mallotus barbatus/53.JPG  
  inflating: Dataset/train/Mallotus barbatus/54.JPG  
  inflating: Dataset/train/Mallotus barbatus/55.JPG  
  inflating: Dataset/train/Mallotus barbatus/56.JPG  
  inflating: Dataset/train/Mallotus barbatus/57.JPG  
  inflating: Dataset/train/Mallotus barbatus/58.JPG  
  inflating: Dataset/train/Mallotus barbatus/59.JPG  
  inflating: Dataset/train/Mallotus barbatus/6.JPG  
  inflating: Dataset/train/Mallot

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load and preprocess your data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # You can adjust the split ratio
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    subset='training'
)

Found 1184 images belonging to 200 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    subset='validation'
)


Found 296 images belonging to 200 classes.


In [18]:
# Create a CNN model
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(200 , activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10
37/37 [==============================] - 500s 13s/step - loss: 1.7948 - accuracy: 0.5329 - val_loss: 14.9400 - val_accuracy: 0.0878
Epoch 2/10
37/37 [==============================] - 260s 7s/step - loss: 0.8425 - accuracy: 0.7796 - val_loss: 28.1107 - val_accuracy: 0.0507
Epoch 3/10
37/37 [==============================] - 266s 7s/step - loss: 0.5927 - accuracy: 0.8336 - val_loss: 41.7747 - val_accuracy: 0.0405
Epoch 4/10
37/37 [==============================] - 271s 7s/step - loss: 0.4889 - accuracy: 0.8556 - val_loss: 23.2805 - val_accuracy: 0.0608
Epoch 5/10
37/37 [==============================] - 257s 7s/step - loss: 0.3805 - accuracy: 0.8860 - val_loss: 32.2015 - val_accuracy: 0.0405
Epoch 6/10
37/37 [==============================] - 296s 8s/step - loss: 0.3737 - accuracy: 0.8927 - val_loss: 31.5315 - val_accuracy: 0.0405
Epoch 7/10
37/37 [==============================] - 270s 7s/step - loss: 0.3503 - accuracy: 0.9122 - val_loss: 31.9572 - val_accuracy: 0.0405
Epoch

In [21]:

# Save the trained model
model.save('medicinal_plant_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
